# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_filmaffinity(title_type=None, user_rating=None, start_date=None, end_date=None):
    url = "https://www.filmaffinity.com/us/topgen.php" # I used other URL to test this exercise
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error accessing the page: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find movie items
    movie_items = soup.find_all('div', class_='mc-title')
    
    if not movie_items:
        print("No movies found on the page.")
        return None

    movie_data = []

    for i, item in enumerate(movie_items, 1):
        title = item.find('a').text.strip()
        year = item.find_next('span').text.strip('()')
        rating = item.find_next('div', class_='rating').text.strip() if item.find_next('div', class_='rating') else None
        description = item.find_next('p').text.strip() if item.find_next('p') else None
        runtime = None
        genre = None
        director = None
        stars = None
        votes = None
        gross = None

        movie_data.append({
            'Movie Nr': i,
            'Title': title,
            'Year': year,
            'Rating': rating,
            'Runtime (min)': runtime,
            'Genre': genre,
            'Description': description,
            'Director': director,
            'Stars': stars,
            'Votes': votes,
            'Gross ($M)': gross
        })

    df = pd.DataFrame(movie_data)
    
    return df

# Example usage of the function
df_movies = scrape_filmaffinity()

# Print the DataFrame to check the output
if df_movies is not None:
    print(df_movies.head())
else:
    print("No data available.")




## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here

import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_filmaffinity(title_type=None, user_rating=None, start_date=None, end_date=None):
    base_url = "https://www.filmaffinity.com/us/topgen.php"
    movie_data = []
    page = 1
    movies_per_page = 50
    total_movies = 631
    
    while (page - 1) * movies_per_page < total_movies:
        params = {
            'title_type': title_type,
            'release_date': f'{start_date},{end_date}',
            'user_rating': f'{user_rating},10',
            'start': (page - 1) * movies_per_page + 1,
            'ref_': 'adv_nxt'
        }
        
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Error accessing the page: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find movie items
        movie_items = soup.find_all('div', class_='mc-title')
        
        if not movie_items:
            print("No more movies found or end of pagination.")
            break

        for i, item in enumerate(movie_items, start=(page - 1) * movies_per_page + 1):
            title = item.find('a').text.strip()
            year = item.find_next('span').text.strip('()')
            rating = item.find_next('div', class_='rating').text.strip() if item.find_next('div', class_='rating') else None
            description = item.find_next('p').text.strip() if item.find_next('p') else None
            runtime = None
            genre = None
            director = None
            stars = None
            votes = None
            gross = None

            movie_data.append({
                'Movie Nr': i,
                'Title': title,
                'Year': year,
                'Rating': rating,
                'Runtime (min)': runtime,
                'Genre': genre,
                'Description': description,
                'Director': director,
                'Stars': stars,
                'Votes': votes,
                'Gross ($M)': gross
            })

        page += 1

    df = pd.DataFrame(movie_data)
    
    return df

# Example usage of the function
df_movies = scrape_filmaffinity()

# Print the DataFrame to check the output
if df_movies is not None:
    print(df_movies.head())
else:
    print("No data available.")


# I was bloqued in filmaffinity.com thanks to this exercise...




